![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


Explore the `students` data using PostgreSQL to find out if you would come to a similar conclusion for international students and see if the length of stay is a contributing factor.

Here is a data description of the columns you may find helpful.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

In [44]:
-- Run this code to view the data in students
SELECT * 
FROM students
LIMIT 5;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24,4,5,Long,3,Average,5,High,,Yes,No,No,No,0,Min,34,23,9,11,8,11,2,27,91,5,5,6,3,2,1,4,1,3,4,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28,5,1,Short,4,High,4,High,,No,No,No,No,2,Min,48,8,7,5,4,3,2,10,39,7,7,7,4,4,4,4,1,1,1,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25,4,6,Long,4,High,4,High,Yes,Yes,No,No,No,2,Min,41,13,4,7,6,4,3,14,51,3,3,3,1,1,2,1,1,1,1,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29,5,1,Short,2,Low,3,Average,No,No,No,No,No,3,Min,37,16,10,10,8,6,4,21,75,5,5,5,5,5,2,2,2,4,4,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28,5,1,Short,1,Low,3,Average,Yes,No,No,No,No,3,Min,37,15,12,5,8,7,4,31,82,5,5,5,2,5,2,5,5,4,4,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No


In [45]:
-- 1. international students summary by stay  

--Return a table with nine rows and five columns.
--The five columns should be aliased as: stay, count_int, average_phq, average_scs, and average_as, in that order.
-- The average columns should contain the average of the todep (PHQ-9 test), tosc (SCS test), and toas (ASISS test) columns for each length of stay, rounded to two decimal places.
--The count_int column should be the number of international students for each length of stay.
-- Sort the results by the length of stay in descending order.

SELECT 
	stay,
	COUNT(*) AS count_int,
	ROUND(AVG(todep),2) AS average_phq, 
	ROUND(AVG(tosc),2) AS average_scs,
	ROUND(AVG(toas),2) AS average_as
FROM students 
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC;

,stay,count_int,average_phq,average_scs,average_as
0,10,1,13.00,32.00,50.00
1,8,1,10.00,44.00,65.00
2,7,1,4.00,48.00,45.00
3,6,3,6.00,38.00,58.67
4,5,1,0.00,34.00,91.00
5,4,14,8.57,33.93,87.71
6,3,46,9.09,37.13,78.00
7,2,39,8.28,37.08,77.67
8,1,95,7.48,38.11,72.80


In [46]:
-- 2. Depression and Stress Summary by Academic Level

-- Return a table with two rows and four columns.
-- The columns should be aliased as: academic, avg_depression, avg_stress, and avg_connectedness.
-- Each row should represent one academic level: undergraduate or graduate.
-- Calculate the average of todep (PHQ-9), toas (ASISS), and tosc (SCS) for each academic level.
-- Round all average values to two decimal places.
-- Sort the rows by avg_depression in descending order.

SELECT 
	academic,
	ROUND(AVG(todep),2) AS avg_depression,
	ROUND(AVG(toas), 2) AS avg_stress,
	ROUND(AVG(tosc), 2) AS avg_connectedness
FROM students
WHERE academic <> ''
GROUP BY academic
ORDER BY avg_depression DESC;

,academic,avg_depression,avg_stress,avg_connectedness
0,Under,8.43,72.07,37.16
1,Grad,5.29,76.05,41.19


In [47]:
-- 2. Language Proficiency and Mental Health

-- Return a table with three rows and four columns.
-- Columns: english_cate, avg_phq, avg_scs, avg_as.
-- Compute the average of todep, tosc, and toas for each English proficiency category.
-- Only include categories that have at least 10 students total.
-- Round all average values to two decimal places.
-- Sort by avg_phq in descending order.

SELECT 
	english_cate, 
	ROUND(AVG(todep),2) AS avg_phq,
	ROUND(AVG(tosc), 2) AS avg_scs,
	ROUND(AVG(toas), 2) AS avg_as
FROM students
WHERE english_cate <> ''
GROUP BY english_cate
HAVING COUNT(*) > 10 
ORDER BY avg_phq DESC;

,english_cate,avg_phq,avg_scs,avg_as
0,Low,9.32,37.05,68.36
1,Average,8.31,38.48,67.44
2,High,7.98,37.05,75.30


In [48]:
-- 3. Stay Duration and Acculturative Stress (International Only)

-- Return a table with the top 7 stay durations (stay) among international students.
-- Columns: stay, total_students, avg_acc_stress.
-- total_students is the number of international students for each stay duration.
-- avg_acc_stress is the average of toas, rounded to two decimal places.
-- Sort the output by avg_acc_stress in ascending order.

SELECT 
	stay, 
	COUNT(*) AS total_students,
	ROUND(AVG(toas),2) AS avg_acc_stress
FROM students 
WHERE inter_dom = 'Inter'
GROUP BY stay 
ORDER BY avg_acc_stress
LIMIT 7;

,stay,total_students,avg_acc_stress
0,7,1,45.00
1,10,1,50.00
2,6,3,58.67
3,8,1,65.00
4,1,95,72.80
5,2,39,77.67
6,3,46,78.00


In [49]:
-- 4. Japanese Proficiency vs Depression

--Return a table with three rows and two columns.
--Columns: japanese_cate, avg_depression.
--For each Japanese language proficiency category, calculate the average of todep (PHQ-9), rounded to two decimal places.
--Only include rows where the average depression score is greater than 5.
--Sort the table by avg_depression in descending order.

SELECT 
	japanese_cate,
	ROUND(AVG(todep),2) AS avg_depression
FROM students
WHERE japanese_cate <> ''
GROUP BY japanese_cate
HAVING ROUND(AVG(todep),2) > 5
ORDER BY avg_depression DESC;

,japanese_cate,avg_depression
0,Average,8.45
1,High,8.17
2,Low,7.95


In [50]:
-- 5. Age Group Analysis for Social Connectedness

--Return a table with five rows and three columns.
--Columns: age_group, count_students, avg_scs.
--Define age_group as follows:
--    '18-22' for age between 18 and 22,
--    '23-26' for age between 23 and 26,
--    '27-30' for age between 27 and 30,
--    '31-35' for age between 31 and 35,
--    '36+' for age > 35.
-- Calculate the number of students and average tosc score per group.
-- Round the average to two decimal places.
-- Sort by count_students in descending order.

SELECT 
	CASE 
		WHEN age BETWEEN 18 AND 22 THEN '18-22'
		WHEN age BETWEEN 23 AND 26 THEN '23-26'
		WHEN age BETWEEN 27 AND 30 THEN '27-30'
		WHEN age BETWEEN 31 AND 35 THEN '31-35'
		ELSE '36+'
	END AS age_group,
	COUNT(*) AS count_students,
	ROUND(AVG(tosc),2) AS avg_scs
FROM students
GROUP BY 
	CASE 
		WHEN age BETWEEN 18 AND 22 THEN '18-22'
		WHEN age BETWEEN 23 AND 26 THEN '23-26'
		WHEN age BETWEEN 27 AND 30 THEN '27-30'
		WHEN age BETWEEN 31 AND 35 THEN '31-35'
		ELSE '36+'
	END
ORDER BY count_students DESC;

,age_group,count_students,avg_scs
0,18-22,217,37.40
1,23-26,32,35.84
2,36+,21,37.33
3,27-30,12,41.17
4,31-35,4,43.50
